In [ ]:
#This notebook is for documentation of the cleaning process. To run the classifier immediately, proceed to the main notebook.

#Use Regex to identify and remove special characters, white space, upper case, links, ect.
import re

def clean_text(s):

    s = s.lower()
    s = re.sub(r"http\S+", "", s)
    s = re.sub(r"<.*?>", "", s)          
    s = re.sub(r"[^a-z0-9\s]", " ", s)   
    s = re.sub(r"\s+", " ", s).strip()

    return s
    
spam['text'] = spam['text'].apply(clean_text)

# Ensure cleaning did not induce NaNs and remove them as needed
nan_mask = spam['text'].isna()
nan_val = spam['text'][spam['text'].isna()]
print(nan_val)

spam = spam.dropna()

#Rebalance dataset with 45% spam as currently seen the real world and to begin reduction of dataset size
spam = spam[spam.label == 1]

ham = spam[spam.label == 0]

desired_spam_fraction = 0.45

desired_spam_count = int(desired_spam_fraction * len(spam))

spam_downsampled = spam.sample(desired_spam_count, random_state=42)

df_balanced = pd.concat([ham, spam_downsampled]).sample(frac=1, random_state=42)

#Remove very long emails, I did this very sparingly since they were more likely to be correlated with Ham, but due to file size limitations it became neccessary to preserve data quantity.¶
spam = spam[spam['text'].str.len() < 75000]

#Convert file to gzip format for compression. These processes significantly reduced dataset size from 130MB down to below 20MB to meet the file size requirement.¶
spam.to_csv('spam.csv.gz', index=False, compression="gzip")